# Prep

In [2]:
import json
import requests
import sys
 
import pymysql

import pandas as pd
import json
import requests
import random
import urllib3
urllib3.disable_warnings()
from tqdm import tqdm
from multiprocess import Pool, Manager, cpu_count
from functools import partial
from ratelimit import sleep_and_retry
import time
import datetime

# Import Data

In [3]:
connection = pymysql.connect(user='music_analyzer_meritus', password='Mer!tu&@123', host='omg-analytics-dev.mysql.database.azure.com', database='streaming_consumption_db'
                                 )
    
cursor = connection.cursor(cursor=pymysql.cursors.DictCursor)

# Read Metadata from Staging Table
cursor.execute("""
select
    song_name_api_opus,
    song_name_opus,
    artist_name_opus,
    catalog_name_opus,
    catalog_type,
    input_parameter,
    country_code,
    song_id_luminate_opus,
    isrc_opus,
    album_name_opus,
    release_date_opus,
    metric_luminate as metric_cat_luminate,
    metric_luminate,
    start_week_id,
    end_week_id,
    song_name_spotify,
    album_name_spotify,
    release_date_spotify,
    isrc_spotify,
    song_id_spotify,
    popularity_score_spotify,
    song_url_spotify,
    artist_name_spotify,
    song_name_genius,
    release_date_genius,
    artist_name_genius,
    song_id_genius,
    available_market,
    explicit,
    popularity,
    acousticness,
    danceability,
    loudness,
    speechiness,
    valence,
    tempo,
    duration_ms,
    instrumentalness,
    key_value,
    liveness,
    mode,
    time_signature
from stg_spotify_genius_detail 
""")
rows = cursor.fetchall()
    
# d_s_g_input_combined dataframe created from stg_spotify_genius_detail MySQL table
d_s_g_input_combined= pd.DataFrame(rows)

connection.commit()
cursor.close()
connection.close()

In [4]:
d_s_g_input_combined.head()

,song_name_api_opus,song_name_opus,artist_name_opus,catalog_name_opus,catalog_type,input_parameter,country_code,song_id_luminate_opus,isrc_opus,album_name_opus,...,loudness,speechiness,valence,tempo,duration_ms,instrumentalness,key_value,liveness,mode,time_signature
0,When It Rains It Pours,When It Rains It Pours,Luke Combs,Show Dog Nashville Metadata Comp Set,Comparison,isrc,US,None,QZ4751600015,This One's for You,...,-5.069,0.0303,0.625,128.027,240760.0,0.000006,6.0,0.2650,1.0,4.0
1,Butterflies,Butterflies,Kacey Musgraves,Show Dog Nashville Metadata Comp Set,Comparison,isrc,US,None,USUM71800130,Golden Hour,...,-6.131,0.0308,0.429,141.955,219053.0,0.000015,4.0,0.1840,0.0,4.0
2,Hard To Love,Hard To Love,Lee Brice,Show Dog Nashville Metadata Comp Set,Comparison,isrc,US,None,USCRB1210131,Hard 2 Love,...,-6.177,0.0332,0.469,103.017,224293.0,0.008080,9.0,0.1890,1.0,4.0
3,Unforgettable,Unforgettable,Thomas Rhett,Show Dog Nashville Metadata Comp Set,Comparison,isrc,US,None,USLXJ1707370,Life Changes,...,-5.497,0.0314,0.828,130.077,157080.0,0.000132,7.0,0.0849,1.0,4.0
4,Break Up In A Small Town,Break Up In A Small Town,Sam Hunt,Show Dog Nashville Metadata Comp Set,Comparison,isrc,US,None,USUM71411195,Montevallo,...,-5.402,0.1740,0.439,136.044,231813.0,0.000000,8.0,0.1580,1.0,4.0


In [5]:
# creating tbl_process_log dataframe to catch edge cases for which details could not be fetched from API
df_tbl_process_log=pd.DataFrame(columns=['song_name_api_opus','song_name_opus','artist_name_opus','catalog_name_opus','catalog_type','input_parameter','country_code','song_id_luminate_opus','isrc_opus','isrc_spotify','Remarks','etl'])
timeout_var = 0
# function for adding songs in log_streaming_ingestion
def tbl_process_log_fn_fact(row,index):
    # series containing required columns data from df_fact record
    df_selected_columns=row.loc[['artist_name_opus',	'song_name_api_opus',	'catalog_name_opus',	'song_id_luminate_opus',	'isrc_opus','input_parameter','country_code',	'catalog_type',	'song_name_opus',	'isrc_spotify']]
    # Converting Series to dataframe and Transposing the dataframe
    df_selected_columns_x=pd.DataFrame(df_selected_columns).T
    #Rearranging the selected columns
    df_selected_columns_x_rearrage=df_selected_columns_x.loc[:,['song_name_api_opus','song_name_opus','artist_name_opus','catalog_name_opus','catalog_type','input_parameter','country_code','song_id_luminate_opus','isrc_opus','isrc_spotify']]
    # adding remarks column in the dataframe record

    df_selected_columns_x_rearrage.at[index,'Remarks']='Not able to fetch song details for fact_luminate_streaming'
    # etl datetime
    now=time.strftime('%Y-%m-%d %H:%M:%S')
    df_selected_columns_x_rearrage.at[index,'etl']=now

    return df_selected_columns_x_rearrage

# function for adding songs in log_streaming_ingestion
def tbl_process_log_fn_dim(row,index):
    # series containg required columns data from df_dim record
    df_selected_columns=row.loc[['artist_name_opus',	'song_name_api_opus',	'catalog_name_opus',	'song_id_luminate_opus',	'isrc_opus','input_parameter','country_code',	'catalog_type',	'song_name_opus',	'isrc_spotify']]
    # Converting Series to dataframe and Transposing the dataframe
    df_selected_columns_x=pd.DataFrame(df_selected_columns).T
    #rearraging the selected columns 
    df_selected_columns_x_rearrage=df_selected_columns_x.loc[:,['song_name_api_opus','song_name_opus','artist_name_opus','catalog_name_opus','catalog_type','input_parameter','country_code','song_id_luminate_opus','isrc_opus','isrc_spotify']]
    # adding remarks column in the dataframe record

    df_selected_columns_x_rearrage.at[index,'Remarks']='Not able to fetch song details for dim_song_detail'
    # etl datetime
    now=time.strftime('%Y-%m-%d %H:%M:%S')
    df_selected_columns_x_rearrage.at[index,'etl']=now
    return df_selected_columns_x_rearrage

# Fetch Data From Luminate API

## Authenticate